In [1]:
# @everywhere used for the implementation of the modules across multiple workers
# Without it, the code does not run

# Add the modules to the ´read´ path of the Julia
push!(LOAD_PATH, "/scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/")


# Custom modules
using Network: HapticNet, VisualNet, train_epoch!, save_as_jld2
using TUM69: NetworkData, kfold
using Utils: a_type, notify!
# Ready-to-use modules
using JLD2, Random
using CUDA


┌ Info: Precompiling Network [top-level]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** increm

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Info: Precompiling TUM69 [top-level]
└ @ Base loading.jl:1278


In [2]:
# Set path
path = CUDA.functional() ? "/userfiles/vaydingul20/data/new" : "data/new" # path of the main data
DATA_PATH = isdir(path) && path
# Set pretrained network path
alexnet_PATH = "/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

"/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

In [14]:
nd = NetworkData(DATA_PATH, "image"; read_type = "normal", read_rate = 0.1, batchsize = 200, atype = a_type(Float32))

Train image data is being loaded!
Test image data is being loaded!


NetworkData(Tuple{String,Int8}[("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5529.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5530.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5531.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5532.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5533.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5534.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5535.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5536.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5537.JPG", 1), ("/userfiles/vaydingul20/data/new/train/image/G1EpoxyRasterPlate/IMG_5538.JPG", 1)  …  ("/userfiles/vaydingul20/data/new/train/image/G9TextileVersion3/IMG_6255.JPG", 69), ("/userfiles/vaydingul20/data/new/train/im

In [15]:
# Seperate into 3 folds for training
kf = kfold(nd; fold=10)
results = []

Any[]

In [ ]:
for (ix, (dtrn, dtst)) in enumerate(kf.folds[1:1])
    

    #Reset the model
    vn = VisualNet(alexnet_PATH; atype=a_type(Float32))

        # Training routine
        res = train_epoch!(vn, dtrn, dtst; progress_bar=true, fig=false, info=true, epoch=2)
        # Save model 
        # Add results to the ´results´vector
        push!(results, res)

    
    save_as_jld2(vn, "vn-$ix.jld2")

end

In [ ]:
# Save accumulated results
JLD2.@save "results_vn.jld2" results = results

In [16]:
dtrn = kf.folds[1][1]
dtst = kf.folds[1][2]

NetworkData(Tuple{String,Int8}[("/userfiles/vaydingul20/data/new/train/image/G7StyrofoamVersion2/IMG_6094.JPG", 53), ("/userfiles/vaydingul20/data/new/train/image/G4CompressedWoodVersion1/IMG_5791.JPG", 24), ("/userfiles/vaydingul20/data/new/train/image/G8PlasticFoilVersion2/IMG_6144.JPG", 58), ("/userfiles/vaydingul20/data/new/train/image/G7FineFoamVersion1/IMG_6021.JPG", 46), ("/userfiles/vaydingul20/data/new/train/image/G1RhombAluminumMesh/IMG_5588.JPG", 5), ("/userfiles/vaydingul20/data/new/train/image/G2CrushedRock/IMG_5624.JPG", 9), ("/userfiles/vaydingul20/data/new/train/image/G1PlasticMeshVersion1/IMG_5554.JPG", 3), ("/userfiles/vaydingul20/data/new/train/image/G9FloorCloth/IMG_6177.JPG", 61), ("/userfiles/vaydingul20/data/new/train/image/G1PlasticMeshVersion1/IMG_5555.JPG", 3), ("/userfiles/vaydingul20/data/new/train/image/G2StoneTileVersion2/IMG_5677.JPG", 14)  …  ("/userfiles/vaydingul20/data/new/train/image/G8PlasticFoilVersion1/IMG_6133.JPG", 57), ("/userfiles/vaydingul20/

In [17]:
vn = VisualNet(alexnet_PATH, atype = a_type(Float32))

VisualNet(Network.GCN((Network.Conv(P(Knet.KnetArrays.KnetArray{Float32,4}(11,11,3,48)), P(Knet.KnetArrays.KnetArray{Float32,4}(1,1,48,1)), Knet.Ops20.relu, 0, (0, 0), (4, 4), (3, 3), (2, 2), true, Knet.KnetArrays.KnetArray{Float32,N} where N), Network.Conv(P(Knet.KnetArrays.KnetArray{Float32,4}(5,5,48,256)), P(Knet.KnetArrays.KnetArray{Float32,4}(1,1,256,1)), Knet.Ops20.relu, 0, (2, 2), (1, 1), (3, 3), (2, 2), true, Knet.KnetArrays.KnetArray{Float32,N} where N), Network.Conv(P(Knet.KnetArrays.KnetArray{Float32,4}(3,3,256,192)), P(Knet.KnetArrays.KnetArray{Float32,4}(1,1,192,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (1, 1), (1, 1), false, Knet.KnetArrays.KnetArray{Float32,N} where N), Network.Conv(P(Knet.KnetArrays.KnetArray{Float32,4}(3,3,192,192)), P(Knet.KnetArrays.KnetArray{Float32,4}(1,1,192,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (1, 1), (1, 1), false, Knet.KnetArrays.KnetArray{Float32,N} where N), Network.Conv(P(Knet.KnetArrays.KnetArray{Float32,4}(3,3,192,256)), P(Knet.KnetArra

In [18]:
res = train_epoch!(vn, dtrn, dtst; progress_bar=true, fig=false, info=false, epoch=10)

┣█▎                  ┫ [6.61%, 109/1650, 16:14/04:05:42, 1.31s/i]  

LoadError: InterruptException: